In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import itertools
import json

In [2]:
data_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/3.data/Keyword/"
src_path = "/home/ai-sjjy/workspace/juyoung/JasoseoAI_project/2.src/Keyword_AI/"

* 이름이 달라서 분리된것 => diff_corpName_raw
* 기업이 없는것 => noCorp_raw
* 취업TIP메뉴가 없는것 => noCorpInfo_raw
* 추출된 것 => wantNValue_raw

In [9]:
with open(data_path+'JobKorea_processed.pickle', 'rb') as f:
    jobkorea = pickle.load(f)

with open(data_path+'/Incruit/diff_corpName_raw.pickle', 'rb') as f:
    diff_corpName_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorp_raw.pickle', 'rb') as f:
    noCorp_raw = pickle.load(f)
    
with open(data_path+'/Incruit/noCorpInfo_raw.pickle', 'rb') as f:
    noCorpInfo_raw = pickle.load(f)
    
with open(data_path+'/Incruit/wantNValue_raw.pickle', 'rb') as f:
    wantNValue_raw = pickle.load(f)

In [10]:
print(len(diff_corpName_raw))
print(len(noCorp_raw))
print(len(noCorpInfo_raw))
print(len(wantNValue_raw))

768
94
146
506


In [12]:
jobkorea.columns

Index(['company_name', 'company_url', 'apply_info', 'applicant_info', 'score',
       'score_desc', 'url', 'Question', 'content', 'comment', 'split_content'],
      dtype='object')

In [13]:
jobkorea = jobkorea[['company_name', 'content']]
jobkorea

,company_name,content
0,(주)경동나비엔,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...
1,(주)포스코인터내셔널,[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...
2,㈜하나은행,"[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강의..."
3,조선비즈,[저는 군대에서 행정 업무를 맡으면서 문제를 해결해 임무수행에 어려움을 덜어낸 경험...
4,㈜CJ ENM,[음악을 사랑합니다. 음악뿐만 아니라 문화산업 전반에 깊은 관심이 있습니다. CJ ...
...,...,...
6626,삼화페인트공업(주),"[남들을 배려하며 착하게 사는 것이 옳다고 배우며 살아왔기 때문에, 이타적인 성향이..."
6627,LG디스플레이,"[2013년 여름, 천호식품에서 아르바이트를 했었습니다. 제품 생산에서부터 포장, ..."
6628,엘에스전선(주),[ 선배들의 추천을 받아 13년도 동아리 회장을 맡았습니다. 당시 숫기가 없어 동기...
6629,S-OIL,[ 최초를 시도한 기업은 많습니다. 하지만 최초보다 더욱 중요한 것이 최고라고 생각...


In [14]:
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))

<ipython-input-14-3d12f3f8d7bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub(" +","",x.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", "")))
<ipython-input-14-3d12f3f8d7bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobkorea['company_name'] = jobkorea['company_name'].apply(lambda x : re.sub("[^가-힣a-z0-9]", "", x))


In [15]:
jobkorea.head(3)

,company_name,content
0,경동나비엔,[새로운 도전을 위해 대학 시절 OO 유학을 떠나게 되었습니다. 조금 늦은 나이의 ...
1,포스코인터내셔널,[어떠한 일을 시작하기 전에 두려운 마음이 들 때마다 이러한 마음가짐을 갖추고 도전...
2,하나은행,"[벤처기업, 제조 공장 자동화, 금융 시스템 개발 경력저는 어려서 배운 컴퓨터 강의..."


In [16]:
group = jobkorea.groupby('company_name')

jobkorea_group = dict()
for idx, g in group:
    sentences = []
    for sentence in g.content.tolist():
        for s in sentence:
            sentences.append(s)
    jobkorea_group[idx] = sentences

In [17]:
len(jobkorea_group)

1513

In [28]:
company_names= list(jobkorea_group.keys())

In [29]:
company_names[0]

'11번가'

In [25]:
wantNValue_raw['11']

KeyError: '11번가'

In [32]:
# 기업 - ([인재상], [핵심가치])
for k, v in wantNValue_raw.items():
    print(k, v)
    print(v[0], v[1])
    break

aj네트웍스 (['따뜻하게', '재미있게', '단순하게', '정직하게'], ['설레임', '귀기울임', '어울림'])
['따뜻하게', '재미있게', '단순하게', '정직하게'] ['설레임', '귀기울임', '어울림']


In [34]:
new_company_infos = dict()
for key, value in wantNValue_raw.items():
    if key in company_names:
        new_company_infos[key] = {"contents" : jobkorea_group[key], "인재상" : value[0], "핵심가치" : value[1]}

In [35]:
len(new_company_infos)

505

In [36]:
with open(data_path+'CompanyKeywords.json', 'w', encoding='utf-8') as f: # 회사-[질문:답변] => dict 형식
    json.dump(new_company_infos, f, indent="\t")

In [38]:
total = 0
for key, value in new_company_infos.items():
    total += len(value['contents'])

In [39]:
total

11673

## 더 크롤링

In [43]:
import requests
import re
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver

In [49]:
with open(data_path+'JobKorea_processed.pickle', 'rb') as f:
    jobkorea = pickle.load(f)

In [53]:
wanted = list(wantNValue_raw.keys())

In [54]:
wanted[0]

'aj네트웍스'

In [44]:
eng = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
kor = ["에이","비","씨","디","이","에프","지","에이치","아이","제이","케이","아이","엠","엔","오","피","큐","알","에스","티","유","브이","더블유","엑스","와이","제트"]

In [63]:
companys = []
for name in set(jobkorea.company_name.tolist()):
    new_name = re.sub(" +","",name.lower().replace('㈜', "").replace("㈔", "").replace("(주)", "").replace("(사)", ""))
    new_name = re.sub("[^가-힣a-z0-9]", "", new_name)
    if new_name not in wanted:
        # [1] 영어 -> 한국어
        eng_str = ""
        for n in name:
            for idx, e in enumerate(eng): 
                if e == n: 
                    eng_str += kor[idx]
                    break
            else: eng_str += n
        # [2] 한국어 -> 영어
        kor_str = ""
        for n in name:
            for idx, k in enumerate(kor): 
                if k == n: 
                    kor_str += eng[idx]
                    break
            else: kor_str += n
        
        companys.append((name, eng_str, kor_str, new_name)) # 원래, 전처리 된 이름

In [64]:
len(companys)

1009

In [65]:
companys[0]

('(주)이엔케이', '(주)이엔케이', '(주)en케e', '이엔케이')

In [66]:
URL = 'https://www.incruit.com/company'
driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(3)

driver.get(url=URL)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
def get_data(corpName):
    enterCount = 0
    wantPeoples = []
    coreValues = []
    
    # 회사 이름으로 찾기
    start_page = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/p/input[1]')
    start_page.send_keys(corpName+'\n')
    time.sleep(0.2)
    enterCount += 1
    
    # 회사 페이지로 들어가기
    corpLinkText = driver.find_elements_by_class_name('listInfo')
    corpLinkTextList = [i.find_element_by_tag_name('a').get_attribute('textContent').strip() for i in corpLinkText]
    
    corp_idx = 0
    for i, cltl in enumerate(corpLinkTextList):
        if corpName in cltl:
            corp_idx = i
            break
    corpLink = corpLinkText[corp_idx].find_element_by_tag_name('a')
    
    # 회사 이름이 있으면 조건 추가
    a_corpName = corpLink.get_attribute('textContent').lower()
    
    if corpName not in a_corpName:
        print("기업이름이 다름니다. 찾으려는 기업: {}, 인크루트의 기업이름: {}".format(corpName, a_corpName))
        for i in range(enterCount):
            driver.back()
        return -1, corpName, a_corpName
    
    corpLink.send_keys('\n')
    time.sleep(0.2)
    enterCount += 1
    
    # 회사 페이지에서 메뉴텝에서 취업TIP이 있나 확인
    menuTap = driver.find_element_by_class_name('tabmenu_temp')
    menuList = menuTap.find_elements_by_tag_name('li')
    menuTextList = [menu.get_attribute('textContent').strip() for menu in menuTap.find_elements_by_tag_name('span')]

    if '취업TIP' in menuTextList:
        idx = menuTextList.index('취업TIP')
        menuList[idx].find_element_by_tag_name('a').send_keys('\n')
        time.sleep(0.2)
        enterCount += 1
    else:
        print(corpName, ": 취업TIP not exist")
        for i in range(enterCount):
            driver.back()
        return -2, corpName, a_corpName
    
    # 취업TIP에서 인재상 찾기 또는 핵심가치 찾기
    contents = driver.find_elements_by_class_name('contPre')
    for i, c in enumerate(contents):
        # 표로 되어 있는 경우
        try:
            rightpeopleClass = c.find_element_by_class_name('gongche_con_rightpeople')
            rightpeopletdTags = rightpeopleClass.find_elements_by_tag_name('td')
            for i in rightpeopletdTags:
                wantPeoples.append(i.get_attribute('textContent'))
        except:
            pass
        try:
            corvalueClass = c.find_element_by_class_name('gongche_con_corevalue')
            corvaluetdTags = corvalueClass.find_elements_by_tag_name('td')
            for i in corvaluetdTags:
                coreValues.append(i.get_attribute('textContent'))
        except:
            headName = c.find_elements_by_tag_name('h3')
            headName = [name.get_attribute('textContent').strip() for name in headName]
            if not(('인재상' in headName) or ('핵심가치' in headName)):
                continue

            headContent = c.find_elements_by_class_name('contPresent')
            headContent = [name.get_attribute('textContent').strip() for name in headContent]

            if '인재상' in headName:
                idx = headName.index('인재상')
                wantPeoples = headContent[idx].strip().split('/')
                wantPeoples = [wantPeople.strip() for wantPeople in wantPeoples]
            else:
                print(corpName, '인재상 not exist')

            if '핵심가치' in headName:
                idx = headName.index('핵심가치')
                coreValues = headContent[idx].strip().split('/')
                coreValues = [coreValue.strip() for coreValue in coreValues]
            else:
                print(corpName, '핵심가치 not exist')
    
    for i in range(enterCount):
        driver.back()
    
    return corpName, wantPeoples, coreValues